In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
import torch

In [3]:
%cd /content/drive/MyDrive/FPSBooster/Model

/content/drive/MyDrive/FPSBooster/Model


In [8]:
train_data = torch.load('train_data.pt')

ModuleNotFoundError: ignored

In [ ]:
train_data.shape

In [ ]:
batch_size = 1
num_epochs = 3
learning_rate = 0.01

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)

In [5]:
data = next(iter(train_loader))

NameError: ignored

In [ ]:
data[0].shape

In [ ]:
from ModelComponents.Criterion import Criterion
from ModelComponents.Model import obtain_model
from ModelComponents.Optimizer import Optimizer

In [ ]:
model = obtain_model()
print(model)

Model(
  (auto_encoder): AutoEncoder(
    (conv_layer_1): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
    (conv_layer_2): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv_layer_3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    (act_layer): ReLU()
    (pool_layer): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (up_conv_layer_1): ConvTranspose2d(128, 64, kernel_size=(2, 2), stride=(2, 2))
    (up_conv_layer_2): ConvTranspose2d(64, 32, kernel_size=(2, 2), stride=(2, 2))
  )
  (up_scale_layer_1): ConvTranspose2d(64, 32, kernel_size=(2, 2), stride=(2, 2))
  (up_scale_layer_2): ConvTranspose2d(128, 32, kernel_size=(4, 4), stride=(4, 4))
  (conv_layer): Conv2d(192, 3, kernel_size=(1, 1), stride=(1, 1))
  (act_layer): ReLU()
)


In [ ]:
optimizer = Optimizer(model, learning_rate)

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    loss = 0.0
    for i, frames in enumerate(train_loader):
        # original shape of frames = (32, 15, 240, 320, 3)
        frames = frames.permute(0, 1, 4, 2, 3)       # permuted shape of frames = (32, 15, 3, 240, 320)     

        # Forward pass
        for j in range(0, frames.shape[1], step = 2):
          if(j < 2):
            frame_1 = frames[:, j, :, : ,:].reshape(frame_1.shape[0], frame)
            output = model(frames[:, j, :, : ,:], frames[:, j + 2, :, : ,:])
            loss = Criterion(output)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
